## Imports

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re

## Import Selenium

In [ ]:
%%capture

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

## Get API

In [ ]:
url = 'https://jovemnerd.com.br/nerdcast/'

get_api = requests.get(url)

text = get_api.text

## Load Soup

In [ ]:
soup = bs(text,  features='html.parser')


## Load Selenium

In [ ]:
driver = webdriver.Chrome(options=options)

driver.get(url)

driver.find_element(By.CLASS_NAME, 'text-button-cookie').click()

## Load List

## Load Page


In [ ]:
driver.find_element(By.ID, 'show-more-podcasts').click()

time.sleep(5)

new_html = driver.page_source

soup = bs(new_html, 'html.parser')

## Loop Podcasts


In [8]:
i = 0

while True:
    button = driver.find_element(By.ID, 'show-more-podcasts')
    button_text = button.text

    get_podcast1 = soup.find_all('a', class_='cat-item product-nerdcast')
    temporary_list = [get_podcast1]

    if temporary_list[0][-1].text != 'NerdCast 01':
      get_podcast_temporary = soup.find_all('a', class_='cat-item product-nerdcast')
      temporary_list.append(get_podcast_temporary)

      button.click()
      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

      i += 1
      print(f'Iteração {i} - {temporary_list[0][-1].text}')

    else:

      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

      get_podcast = soup.find_all('a', class_='cat-item product-nerdcast')
      get_podcast_name = soup.find_all('h2', class_='title')
      get_podcast_duration = soup.find_all('div', class_='info')

      list.extend(get_podcast)
      list2.extend(get_podcast_name)
      list3.extend(get_podcast_duration)

      print(f'Coleta concluída, com o arquivo {list[-1].text}!')
      break

Iteração 1 - Caneca de Mamicas 124
Iteração 2 - Caneca de Mamicas 120
Iteração 3 - Papo de Parceiro 24
Iteração 4 - Papo de Parceiro 23
Iteração 5 - Caneca de Mamicas 107
Iteração 6 - NerdCast 879
Iteração 7 - NerdTech 86
Iteração 8 - Papo de Parceiro 20 
Iteração 9 - Mau Acompanhado 21
Iteração 10 - Lá do Bunker Oscar 02
Iteração 11 - Papo de Parceiro 18
Iteração 12 - Lá do Bunker 75
Iteração 13 - Vai te Catar 15
Iteração 14 - Lá do Bunker 71
Iteração 15 - Extra Oferecido por Magic: The Gathering
Iteração 16 - NerdCast 847
Iteração 17 - Lá do Bunker 56
Iteração 18 - NerdCash 49
Iteração 19 - NerdCast 832
Iteração 20 - NerdCast 827
Iteração 21 - NerdCast 822
Iteração 22 - Lá do Bunker 31
Iteração 23 - Caneca de Mamicas 38
Iteração 24 - Lá do Bunker 22
Iteração 25 - Extra Oferecido por Arcane: League of Legends
Iteração 26 - Lá do Bunker 13
Iteração 27 - Papo de Parceiro 02
Iteração 28 - NerdCast 791
Iteração 29 - Caneca de Mamicas 11
Iteração 30 - Caneca de Mamicas 04
Iteração 31 - Ner

### Get Themes and Subjects

In [9]:
get_themes = soup.find('select', class_='nerdcast-filter')

for item in get_themes:
  theme = item.get_text().strip()

  if theme != '':
    list_themes.append(theme)

list_themes.pop(0)
print(list_themes)

['Animação', 'Artes Marciais', 'Audio Drama', 'Biografia', 'Ciências', 'Cinema', 'Comportamento', 'Cotidiano', 'Dia dos Namorados', 'Educação', 'Empreendedorismo', 'Entrevistas', 'Esportes', 'Game of Thrones', 'Games', 'Gastronomia', 'História', 'História Alternativa', 'Internet', 'Lendas e Mistérios', 'Literatura', 'Lost', 'MAIS POPULARES', 'Melhores Momentos', 'Música', 'Nostalgia', 'Notícias', 'Política', 'Profissões', 'Publicidade', 'Quadrinhos', 'Retrospectiva', 'RPG', 'Séries', 'Star Wars', 'Tecnologia', 'Teoria da Conspiração', 'Terror', 'Trabalho', 'TV', 'Viagens']


In [10]:
for item in list_themes:
  subjects.append([])

print(subjects)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [11]:
list_podcasts_subject = dict(zip(list_themes, subjects))

print(list_podcasts_subject)

{'Animação': [], 'Artes Marciais': [], 'Audio Drama': [], 'Biografia': [], 'Ciências': [], 'Cinema': [], 'Comportamento': [], 'Cotidiano': [], 'Dia dos Namorados': [], 'Educação': [], 'Empreendedorismo': [], 'Entrevistas': [], 'Esportes': [], 'Game of Thrones': [], 'Games': [], 'Gastronomia': [], 'História': [], 'História Alternativa': [], 'Internet': [], 'Lendas e Mistérios': [], 'Literatura': [], 'Lost': [], 'MAIS POPULARES': [], 'Melhores Momentos': [], 'Música': [], 'Nostalgia': [], 'Notícias': [], 'Política': [], 'Profissões': [], 'Publicidade': [], 'Quadrinhos': [], 'Retrospectiva': [], 'RPG': [], 'Séries': [], 'Star Wars': [], 'Tecnologia': [], 'Teoria da Conspiração': [], 'Terror': [], 'Trabalho': [], 'TV': [], 'Viagens': []}


### Loop Themes and Subjects

In [12]:
i_theme = 1
index_theme = 0
list_themes = []

while len(list_podcasts_subject[list_themes[-1]]) <= 0:
  select = driver.find_element(By.CLASS_NAME, 'nerdcast-filter')
  choosed = Select(select).select_by_index(i_theme)

  button = driver.find_element(By.ID, 'show-more-podcasts')

  style_value = button.get_attribute('style')

  if style_value == 'display: block;':

      button.click()
      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

  else:
    time.sleep(5)
    new_html = driver.page_source
    soup = bs(new_html, 'html.parser')

    get_podcast = soup.find_all('a', class_='cat-item product-nerdcast')

    list_podcasts_subject[list_themes[index_theme]] = get_podcast
    print(f'{list_themes[index_theme]}: {list_podcasts_subject[list_themes[index_theme]]}')

    i_theme += 1
    index_theme += 1

Animação: [<a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/nerdcast/la-do-bunker/nepotismo-e-brotheragem/">Lá do Bunker 122</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/nerdcast/caneca-de-mamicas/vizinhas-do-mickey/">Caneca de Mamicas 132</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/nerdcast/gen-v-drama-adolescente-com-poderes-muito-sangue-e-p0/">NerdCast 907</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/nerdcast/la-do-bunker/foi-mal-tava-doidao/">Lá do Bunker 121</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/playlist/especial-rpg-cronicas-de-ghanor/">2011 - 2012 - 2022 - 2023</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/playlist/rpg-call-of-cthulhu/">2017 - 2020</a>, <a class="cat-item product-nerdcast" href="https://jovemnerd.com.br/playlist/rpg-cyberpunk/">2013 - 2016</a>, <a class="cat-item product-nerdcast" href="https://jove

### Get text

In [13]:
for key, value in list_podcasts_subject.items():
    names_list = []
    for item in value:
        text = item.get_text()
        names_list.append(text)
    list_podcasts_subject[key] = names_list[7:]

# Print or use list_podcasts_names as needed
print(list_podcasts_subject)

{'Animação': ['Caneca de Mamicas 123', 'NerdCast 885', 'Lá do Bunker 57', 'Lá do Bunker 53', 'Lá do Bunker 50', 'Lá do Bunker 49', 'Lá do Bunker 47', 'Lá do Bunker 46', 'NerdCast 832', 'Lá do Bunker 38', 'NerdCast 815', 'Extra Oferecido por Arcane: League of Legends', 'Extra Oferecido por Arcane: League of Legends', 'Extra Oferecido por Arcane: League of Legends', 'NerdCast 590', 'NerdCast 563', 'NerdCast 252', 'NerdCast 121', 'NerdCast 81', 'NerdCast 75'], 'Artes Marciais': ['Lá do Bunker 57', 'Lá do Bunker 50', 'NerdCast 309', 'NerdCast 134'], 'Audio Drama': ['NerdCast 898', 'NerdCast 897', 'Lá do Bunker 57', 'Extra Oferecido por Spotify', 'NerdCast 342'], 'Biografia': ['Caneca de Mamicas 121', 'Papo de Parceiro 13', 'Lá do Bunker 57', 'Caneca de Mamicas 67', 'Caneca de Mamicas 66', 'Empreendedor 89', 'Generacast 10', 'NerdCast 807', 'Empreendedor 82', 'NerdCast 728', 'NerdCast 726', 'NerdTech 46', 'NerdCast 676', 'NerdCast 665', 'NerdCast 646', 'NerdCast 344', 'NerdCast 280', 'NerdC

## Data Output

###df

In [168]:
df = pd.DataFrame(columns=['Podcast', 'Link'])

In [169]:
for item in list:
  df.loc[df.shape[0]] = [item.text.strip(), item['href']]

In [170]:
df = df[df['Podcast'] != '2011 - 2012 - 2022 - 2023']
df = df[df['Podcast'] != '2017 - 2020']
df = df[df['Podcast'] != '2013 - 2016']
df = df[df['Podcast'] != 'NerdCast 549 Teaser']

In [171]:
linhas_duplicadas = df[df.duplicated(keep=False)]

print(linhas_duplicadas)

                   Podcast                                               Link
18        Lá do Bunker 118  https://jovemnerd.com.br/nerdcast/la-do-bunker...
19   Caneca de Mamicas 128  https://jovemnerd.com.br/nerdcast/caneca-de-ma...
20            NerdCast 903  https://jovemnerd.com.br/nerdcast/ahsoka-e-con...
21        Lá do Bunker 117  https://jovemnerd.com.br/nerdcast/la-do-bunker...
22   Caneca de Mamicas 127  https://jovemnerd.com.br/nerdcast/caneca-de-ma...
23        Lá do Bunker 118  https://jovemnerd.com.br/nerdcast/la-do-bunker...
24   Caneca de Mamicas 128  https://jovemnerd.com.br/nerdcast/caneca-de-ma...
25            NerdCast 903  https://jovemnerd.com.br/nerdcast/ahsoka-e-con...
26        Lá do Bunker 117  https://jovemnerd.com.br/nerdcast/la-do-bunker...
27   Caneca de Mamicas 127  https://jovemnerd.com.br/nerdcast/caneca-de-ma...
730           NerdCast 692  https://jovemnerd.com.br/nerdcast/era-uma-vez-...
731           NerdCast 692  https://jovemnerd.com.br/nerdcast/er

In [172]:
df = df.drop_duplicates()

In [173]:
df.head()

,Podcast,Link
0,Lá do Bunker 122,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,Caneca de Mamicas 132,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,NerdCast 907,https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,Lá do Bunker 121,https://jovemnerd.com.br/nerdcast/la-do-bunker...
7,Caneca de Mamicas 131,https://jovemnerd.com.br/nerdcast/caneca-de-ma...


### Add themes to each podcast

In [174]:
df.insert(len(df.columns)-1, column = 'Theme', value = '')

In [175]:
podcast_names = []

for item in list:
  podcast_names.append(item.text)

print(podcast_names)

['Lá do Bunker 122', 'Caneca de Mamicas 132', 'NerdCast 907', 'Lá do Bunker 121', '2011 - 2012 - 2022 - 2023', '2017 - 2020', '2013 - 2016', 'Caneca de Mamicas 131', 'NerdCast 906', 'NerdTech 93', 'Lá do Bunker 120', 'Caneca de Mamicas 130', 'NerdCast 905', 'Papo de Parceiro 27', 'Lá do Bunker 119', 'Caneca de Mamicas 129', 'NerdCast 904', 'Extra Oferecido por SomosCoop', 'Lá do Bunker 118', 'Caneca de Mamicas 128', 'NerdCast 903', 'Lá do Bunker 117', 'Caneca de Mamicas 127', 'Lá do Bunker 118', 'Caneca de Mamicas 128', 'NerdCast 903', 'Lá do Bunker 117', 'Caneca de Mamicas 127', 'NerdCast 902', 'NerdTech 92', 'Lá do Bunker 116', 'Caneca de Mamicas 126', 'NerdCast 901', 'Lá do Bunker 115', 'Caneca de Mamicas 125', 'NerdCast 900', 'Papo de Parceiro 26', 'Lá do Bunker 114', 'Caneca de Mamicas 124', 'NerdCast 899', 'NerdCast EXTRA', 'Lá do Bunker 113', 'Caneca de Mamicas 123', 'NerdCast 898', 'Extra Oferecido por Prime Vídeo', 'Lá do Bunker 112', 'Caneca de Mamicas 122', 'NerdCast 897', '

In [176]:
podcast_names_themes = []

for key, value in list_podcasts_subject.items():
  for item in value:
    podcast_names_themes.append((key, item))
print(podcast_names_themes)

[('Animação', 'Caneca de Mamicas 123'), ('Animação', 'NerdCast 885'), ('Animação', 'Lá do Bunker 57'), ('Animação', 'Lá do Bunker 53'), ('Animação', 'Lá do Bunker 50'), ('Animação', 'Lá do Bunker 49'), ('Animação', 'Lá do Bunker 47'), ('Animação', 'Lá do Bunker 46'), ('Animação', 'NerdCast 832'), ('Animação', 'Lá do Bunker 38'), ('Animação', 'NerdCast 815'), ('Animação', 'Extra Oferecido por Arcane: League of Legends'), ('Animação', 'Extra Oferecido por Arcane: League of Legends'), ('Animação', 'Extra Oferecido por Arcane: League of Legends'), ('Animação', 'NerdCast 590'), ('Animação', 'NerdCast 563'), ('Animação', 'NerdCast 252'), ('Animação', 'NerdCast 121'), ('Animação', 'NerdCast 81'), ('Animação', 'NerdCast 75'), ('Artes Marciais', 'Lá do Bunker 57'), ('Artes Marciais', 'Lá do Bunker 50'), ('Artes Marciais', 'NerdCast 309'), ('Artes Marciais', 'NerdCast 134'), ('Audio Drama', 'NerdCast 898'), ('Audio Drama', 'NerdCast 897'), ('Audio Drama', 'Lá do Bunker 57'), ('Audio Drama', 'Ext

In [177]:
print(list_podcasts_subject)

{'Animação': ['Caneca de Mamicas 123', 'NerdCast 885', 'Lá do Bunker 57', 'Lá do Bunker 53', 'Lá do Bunker 50', 'Lá do Bunker 49', 'Lá do Bunker 47', 'Lá do Bunker 46', 'NerdCast 832', 'Lá do Bunker 38', 'NerdCast 815', 'Extra Oferecido por Arcane: League of Legends', 'Extra Oferecido por Arcane: League of Legends', 'Extra Oferecido por Arcane: League of Legends', 'NerdCast 590', 'NerdCast 563', 'NerdCast 252', 'NerdCast 121', 'NerdCast 81', 'NerdCast 75'], 'Artes Marciais': ['Lá do Bunker 57', 'Lá do Bunker 50', 'NerdCast 309', 'NerdCast 134'], 'Audio Drama': ['NerdCast 898', 'NerdCast 897', 'Lá do Bunker 57', 'Extra Oferecido por Spotify', 'NerdCast 342'], 'Biografia': ['Caneca de Mamicas 121', 'Papo de Parceiro 13', 'Lá do Bunker 57', 'Caneca de Mamicas 67', 'Caneca de Mamicas 66', 'Empreendedor 89', 'Generacast 10', 'NerdCast 807', 'Empreendedor 82', 'NerdCast 728', 'NerdCast 726', 'NerdTech 46', 'NerdCast 676', 'NerdCast 665', 'NerdCast 646', 'NerdCast 344', 'NerdCast 280', 'NerdC

In [178]:
podcast_names_themes[0]

('Animação', 'Caneca de Mamicas 123')

In [179]:
for item in podcast_names_themes:
  for name in df['Podcast']:
    if item == name:
      df['Theme'] = item

In [180]:
df.tail()

,Podcast,Theme,Link
1571,NerdCast 05,,https://jovemnerd.com.br/nerdcast/nerdcast-05-...
1572,NerdCast 04,,https://jovemnerd.com.br/nerdcast/04-rocky-bal...
1573,NerdCast 03,,https://jovemnerd.com.br/nerdcast/03-quadrinho...
1574,NerdCast 02,,https://jovemnerd.com.br/nerdcast/02-batman-co...
1575,NerdCast 01,,https://jovemnerd.com.br/nerdcast/01-super-hom...


In [181]:
for item in podcast_names_themes:
  mask = df['Podcast'] == item[1]

  if not df.loc[mask, 'Theme'].empty:
    existing_theme = df.loc[mask, 'Theme'].iloc[0]
    updated_theme = ', '.join([existing_theme, item[0]])
    df.loc[mask, 'Theme'] = updated_theme

In [182]:
len(df["Theme"][34])

15

In [183]:
df["Theme"][34]

', Comportamento'

# Corrigir

In [184]:
list_of_numbers = []

for item in df['Podcast']:

  this = any(char.isdigit() for char in item)

  if this is False:
    item.index
    result_string = ''.join(item)

    list_of_numbers.append(result_string)

  else:
    pass

In [185]:
for item in list_of_numbers:
  df = df[df['Podcast'] != item]

In [186]:
list_text = []

for item in df['Podcast']:
  this = re.split(r'[0-9]', item)
  list_text.extend(this)

In [187]:
list_number = []

for item in df['Podcast']:
  this = re.split(r'[a-z A-Z á-ú]', item)

  list_number.extend(this)

In [188]:
text_new = []

for item in list_text:
  if len(item) >= 2:
    this = item.strip()
    text_new.append(this)
  else:
    pass

In [189]:
number_new = []

number_new = [item for item in list_number if item != '']

In [190]:
df.insert(len(df.columns) - 1, column = 'Numero', value = 0)

In [191]:
df['Numero'] = number_new
df['Podcast'] = text_new

In [192]:
df.insert(0, 'ID', 1)
df['ID'] = range(1, len(df) + 1)

In [193]:
df[50:100]

,ID,Podcast,Theme,Numero,Link
61,51,NerdCast,", Cotidiano",893,https://jovemnerd.com.br/nerdcast/mais-regras-...
62,52,NerdTech,", Internet, Tecnologia",90,https://jovemnerd.com.br/nerdcast/nerdtech/rin...
63,53,Lá do Bunker,", Notícias",107,https://jovemnerd.com.br/nerdcast/la-do-bunker...
64,54,Caneca de Mamicas,", Cinema",117,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
65,55,NerdCast,", RPG",892,https://jovemnerd.com.br/nerdcast/rpg-ghanor-5...
66,56,Lá do Bunker,", Notícias",106,https://jovemnerd.com.br/nerdcast/la-do-bunker...
67,57,Caneca de Mamicas,", Esportes",116,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
68,58,NerdCast,", Ciências, História",891,https://jovemnerd.com.br/nerdcast/oppenheimer-...
69,59,Papo de Parceiro,", Empreendedorismo",24,https://jovemnerd.com.br/nerdcast/papo-de-parc...
70,60,Lá do Bunker,", Notícias",105,https://jovemnerd.com.br/nerdcast/la-do-bunker...


###df2

In [194]:
df2 = pd.DataFrame(columns=['Assunto', 'Link'])

In [195]:
for item in list2:
  df2.loc[df2.shape[0]] = [item.text.strip(), item.find('a')['href']]

In [196]:
linhas_duplicadas = df2[df2.duplicated(keep=False)]

print(linhas_duplicadas)

                              Assunto  \
15               Gênesis de Miranhete   
16   Quando a gente ama a gente cuida   
17            Ahsoka e… …contemplação   
18              Se fazendo de difícil   
19                    O duelo do amor   
20               Gênesis de Miranhete   
21   Quando a gente ama a gente cuida   
22            Ahsoka e… …contemplação   
24              Se fazendo de difícil   
25                    O duelo do amor   
799         Era uma vez um… Tarantino   
800         Era uma vez um… Tarantino   

                                                  Link  
15   https://jovemnerd.com.br/nerdcast/la-do-bunker...  
16   https://jovemnerd.com.br/nerdcast/caneca-de-ma...  
17   https://jovemnerd.com.br/nerdcast/ahsoka-e-con...  
18   https://jovemnerd.com.br/nerdcast/la-do-bunker...  
19   https://jovemnerd.com.br/nerdcast/caneca-de-ma...  
20   https://jovemnerd.com.br/nerdcast/la-do-bunker...  
21   https://jovemnerd.com.br/nerdcast/caneca-de-ma...  
22   https

In [197]:
df2 = df2.drop_duplicates()

In [198]:
linhas_duplicadas = df2[df2.duplicated(keep=False)]

print(linhas_duplicadas)

Empty DataFrame
Columns: [Assunto, Link]
Index: []


In [199]:
df2.insert(0, 'ID', 1)
df2['ID'] = range(1, len(df2) + 1)

<ipython-input-199-55a6c10cc2fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ID'] = range(1, len(df2) + 1)


In [200]:
df2.head()

,ID,Assunto,Link
0,1,Nepotismo e brotheragem,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,2,Vizinhas do Mickey,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,3,"GEN V: Drama adolescente com poderes, muito sa...",https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,4,"Foi mal, tava doidão",https://jovemnerd.com.br/nerdcast/la-do-bunker...
4,5,Só sei que NASA sei,https://jovemnerd.com.br/nerdcast/caneca-de-ma...


###df3

In [201]:
df3 = pd.DataFrame(columns=['Duracao', 'Data_Lancamento', 'Link'])

In [202]:
for item in list3:
  if item.find('time'):
    duracao = item.find('time').get_text().split('•')[0][1:].strip()
    list_duration.append(duracao)

    data_lancamento = item.find('time').get_text().split('•')[1][1:].strip()
    list_launch.append(data_lancamento)

    link = item.find('a')['href']
    list_link.append(link)

  else:
    pass

In [203]:
df3['Duracao'] = list_duration
df3['Data_Lancamento'] = list_launch
df3['Link'] = list_link

In [204]:
linhas_duplicadas = df3[df3.duplicated(keep=False)]

print(linhas_duplicadas)

                  Duracao         Data_Lancamento  \
0      1 hora e 0 minutos  14 de novembro de 2023   
1     1 hora e 16 minutos  11 de novembro de 2023   
2     1 hora e 53 minutos  10 de novembro de 2023   
3              38 minutos   7 de novembro de 2023   
4              58 minutos   4 de novembro de 2023   
...                   ...                     ...   
4981           16 minutos      2 de abril de 2006   
4982           15 minutos      2 de abril de 2006   
4983           21 minutos      2 de abril de 2006   
4984           11 minutos      2 de abril de 2006   
4985           13 minutos      2 de abril de 2006   

                                                   Link  
0     https://jovemnerd.com.br/nerdcast/la-do-bunker...  
1     https://jovemnerd.com.br/nerdcast/caneca-de-ma...  
2     https://jovemnerd.com.br/nerdcast/gen-v-drama-...  
3     https://jovemnerd.com.br/nerdcast/la-do-bunker...  
4     https://jovemnerd.com.br/nerdcast/caneca-de-ma...  
...            

In [205]:
df3 = df3.drop_duplicates()

In [206]:
linhas_duplicadas = df[df.duplicated(keep=False)]

print(linhas_duplicadas)

Empty DataFrame
Columns: [ID, Podcast, Theme, Numero, Link]
Index: []


In [207]:
df3.head(30)

,Duracao,Data_Lancamento,Link
0,1 hora e 0 minutos,14 de novembro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,1 hora e 16 minutos,11 de novembro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,1 hora e 53 minutos,10 de novembro de 2023,https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,38 minutos,7 de novembro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
4,58 minutos,4 de novembro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
5,1 minuto,3 de novembro de 2023,https://jovemnerd.com.br/nerdcast/bastidores-d...
6,57 minutos,3 de novembro de 2023,https://jovemnerd.com.br/nerdcast/nerdtech/o-q...
7,56 minutos,31 de outubro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
8,1 hora e 12 minutos,28 de outubro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
9,1 hora e 52 minutos,27 de outubro de 2023,https://jovemnerd.com.br/nerdcast/qual-e-a-pau...


### df4

In [208]:
df4 = pd.DataFrame(list_podcasts_subject.items(), columns=['Theme', 'Podcasts'])

In [209]:
df4.head()

,Theme,Podcasts
0,Animação,"[Caneca de Mamicas 123, NerdCast 885, Lá do Bu..."
1,Artes Marciais,"[Lá do Bunker 57, Lá do Bunker 50, NerdCast 30..."
2,Audio Drama,"[NerdCast 898, NerdCast 897, Lá do Bunker 57, ..."
3,Biografia,"[Caneca de Mamicas 121, Papo de Parceiro 13, L..."
4,Ciências,"[Caneca de Mamicas 130, NerdCast 896, NerdCast..."


# Out Put

In [210]:
output_file = "dados_coletados.csv"

df.to_csv(output_file, index=False)

In [211]:
output_file = "dados_coletados2.csv"

df2.to_csv(output_file, index=False)

In [212]:
output_file = "dados_coletados3.csv"

df3.to_csv(output_file, index=False)

In [213]:
output_file = "dados_coletados4.csv"

df4.to_csv(output_file, index=False)

In [214]:
df.head()

,ID,Podcast,Theme,Numero,Link
0,1,Lá do Bunker,,122,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,2,Caneca de Mamicas,,132,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,3,NerdCast,,907,https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,4,Lá do Bunker,,121,https://jovemnerd.com.br/nerdcast/la-do-bunker...
7,5,Caneca de Mamicas,", Comportamento",131,https://jovemnerd.com.br/nerdcast/caneca-de-ma...


In [215]:
df2.head()

,ID,Assunto,Link
0,1,Nepotismo e brotheragem,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,2,Vizinhas do Mickey,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,3,"GEN V: Drama adolescente com poderes, muito sa...",https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,4,"Foi mal, tava doidão",https://jovemnerd.com.br/nerdcast/la-do-bunker...
4,5,Só sei que NASA sei,https://jovemnerd.com.br/nerdcast/caneca-de-ma...


In [216]:
df3.head()

,Duracao,Data_Lancamento,Link
0,1 hora e 0 minutos,14 de novembro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
1,1 hora e 16 minutos,11 de novembro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
2,1 hora e 53 minutos,10 de novembro de 2023,https://jovemnerd.com.br/nerdcast/gen-v-drama-...
3,38 minutos,7 de novembro de 2023,https://jovemnerd.com.br/nerdcast/la-do-bunker...
4,58 minutos,4 de novembro de 2023,https://jovemnerd.com.br/nerdcast/caneca-de-ma...
